In [ ]:
# Automatically reload files when they are changed so there is no need to restart the kernel.
%load_ext autoreload
%autoreload 2

import torch
from torch import nn
import numpy as np
import random
import matplotlib.pyplot as plt
import pickle

torch.backends.cudnn.deterministic = True

# This is the seed that I used to get the reported accuracy levels
seed = 2023
def reset_seed():
	torch.manual_seed(seed)
	random.seed(seed)
	np.random.seed(seed)

def cuda_get_device():
	is_cuda = torch.cuda.is_available()
	if is_cuda:
		return torch.device("cuda")
	else:
		print("GPU not available, CPU used")
		return torch.device("cpu")

# Define the model class
class GRUWithConvLayer(nn.Module):
	def __init__(self, input_size, output_size, hidden_dim, conv_dropout, fc_dropout, second_layer=False):
		super(GRUWithConvLayer, self).__init__()
		self.hidden_dim = hidden_dim
		kernel_size = 5
		padding = 0
		self.convlayer = nn.Conv2d(1, 1, (kernel_size, kernel_size), stride=1, padding=padding)
		self.batchnorm = nn.BatchNorm2d(1)
		self.conv_dropout = nn.Dropout(p=conv_dropout)
		self.gru = nn.GRU(input_size=input_size - (kernel_size - 1) + 2*padding, hidden_size=hidden_dim, batch_first=True)
		self.fc_dropout = nn.Dropout(p=fc_dropout)
		self.second_layer = second_layer
		if second_layer:
			intermediate_dim = 100
			self.fc1= nn.Linear(hidden_dim, intermediate_dim)
			self.fc2 = nn.Linear(intermediate_dim, output_size)
		else:
			self.fc1 = nn.Linear(hidden_dim, output_size)
	
	def forward(self, x):
		after_conv = self.batchnorm(self.convlayer(x)).squeeze(1)
		after_conv_drop = self.conv_dropout(after_conv)
		hidden = self.gru(after_conv_drop)[0]
		if self.second_layer:
			out = self.fc1(hidden)
			out = self.fc_dropout(out)
			out = self.fc2(out)
		else:
			out = self.fc1(hidden)
		return out

In [ ]:
%matplotlib widget

reset_seed()